In [1]:
import yaml
import wandb

config_path = "configs/base_config.yaml"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# wandb.init(project=config['wandb']['project'],
#            entity=config['wandb']['entity'],
#            config=config)

In [2]:
import torchreid
datamanager = torchreid.data.ImageDataManager(
    root            = config['data']['root'],
    sources         = config['data']['sources'],
    targets         = config['data']['targets'],
    height          = config['data']['height'],
    width           = config['data']['width'],
    batch_size_train= config['data']['batch_size'],
    batch_size_test = config['data']['batch_size'],
    transforms      = config['data']['transforms']
)

/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

In [3]:
model = torchreid.models.build_model(
    name        = config['model']['name'],
    num_classes = 2000 if 'tango2000' in config['data']['sources'] else datamanager.num_train_pids, #2000 for Tango
    loss        = config['loss']['name'],
    pretrained  = config['model']['pretrained'],
    pooling     = config['model']['final_pooling'],
    **config['custom']
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim=config['train']['optim'],
    lr=config['train']['lr']
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    **config['train']['lr_scheduler'],
)

gem


In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [7]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=config['loss']['softmax']['label_smooth'],
    # logger=wandb
)

In [8]:
engine.run(
    save_dir=f"log/{config['model']['name']}_{config['data']['sources'][0]}",
    max_epoch=60,
    eval_freq=2,
    print_freq=50,
    test_only=False,
    visrank=False,
    eval_metric='default'
)

=> Start training
epoch: [1/60][50/101]	time 0.151 (0.185)	data 0.000 (0.005)	eta 0:18:33	loss 6.4001 (6.7094)	acc 0.7812 (0.7500)	lr 0.001500
epoch: [1/60][100/101]	time 0.151 (0.168)	data 0.000 (0.003)	eta 0:16:42	loss 5.9668 (6.4520)	acc 2.3438 (1.0625)	lr 0.001500
epoch: [2/60][50/101]	time 0.151 (0.154)	data 0.000 (0.003)	eta 0:15:10	loss 5.5624 (5.7185)	acc 3.1250 (3.2969)	lr 0.001500
epoch: [2/60][100/101]	time 0.150 (0.152)	data 0.000 (0.002)	eta 0:14:53	loss 4.9390 (5.5046)	acc 11.7188 (5.2656)	lr 0.001500
##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-2048 matrix
Speed: 0.0022 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
[   0  455    0 ... 1026  668  723] <class 'numpy.ndarray'> False
** Results **
mAP: 4.8%
CMC curve
Rank-1  : 9.6%
Rank-5  : 21.4%
Rank-10 : 29.0%
Rank-20 : 39.1%
Checkpoint saved to 